In [1]:
import os
import json
import cv2
import numpy as np
import tensorflow.compat.v1 as tf
from run_case import real_model 
# This is the "Magic Sauce" for M4 Pro
tf.disable_eager_execution()
tf.disable_v2_behavior()
tf.disable_v2_behavior()
tf.reset_default_graph() # Start with a clean slate

# Manual mapping to bypass Pylance resolution errors
placeholder = tf.placeholder
variable_scope = tf.variable_scope

# M4 PRO GPU CONFIG
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    try:
        for device in gpu_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("🚀 M4 Pro GPU (Metal) Ready")
    except Exception as e:
        print(f"⚠️ GPU setup info: {e}")

tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term
🚀 M4 Pro GPU (Metal) Ready


In [2]:

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
with tf.variable_scope("test"):
    p = tf.placeholder(tf.float32, shape=[1])
    print("✅ Success! TensorFlow 1.x logic is working.")

✅ Success! TensorFlow 1.x logic is working.


In [3]:
def chair_data_generator(json_path, batch_size=1):
    with open(json_path, 'r') as f:
        data = json.load(f).get('chair', [])
    
    while True:
        np.random.shuffle(data)
        for i in range(0, len(data), batch_size):
            batch_data = data[i:i + batch_size]
            b_imgs, b_voxels, b_masks = [], [], []
            
            for entry in batch_data:
                # Load Image
                img = cv2.imread(entry['image_path'])
                if img is None: continue
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (128, 128)) / 255.0
                
                # Load Voxels
                voxels = np.load(entry['voxel_gt_path']).astype(np.float32)
                voxels = voxels.transpose(2, 1, 0)
                if voxels.ndim == 4: voxels = np.squeeze(voxels)
                
                # Load Mask (New)
                # Assuming entry['mask_path'] exists in your JSON
                mask = cv2.imread(entry['mask_path'], cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (32, 32)) # Match voxel spatial grid
                mask = (mask > 127).astype(np.float32) # Binary 0 or 1
                
                b_imgs.append(img)
                b_voxels.append(voxels)
                b_masks.append(mask)
            
            if len(b_imgs) == batch_size:
                yield np.array(b_imgs), np.array(b_voxels), np.array(b_masks)

In [5]:
JSON_PATH = "pascal_metadata_training_fixed.json"
EPOCHS = 5
STEPS_PER_EPOCH = 200 
BATCH_SIZE = 1

save_path = "./finetuned_chair_with_masks_model_learning_rate_1e_4.cptk"
gen = chair_data_generator(JSON_PATH, BATCH_SIZE)
learning_rate = 1e-4


In [6]:
with open("pascal_metadata_training_fixed.json", 'r') as f:
    data = json.load(f).get('chair', [])
# Check the first 5 samples
for i in range(min(5, len(data))):
    voxel_path = data[i]['voxel_gt_path']
    if os.path.exists(voxel_path):
        voxels = np.load(voxel_path)
        
        print(f"File: {os.path.basename(voxel_path)}")
        print(f"  - Shape: {voxels.shape}")
        print(f"  - Dtype: {voxels.dtype}")
        print(f"  - Min Value: {np.min(voxels)}")
        print(f"  - Max Value: {np.max(voxels)}")
        print(f"  - Unique Values: {np.unique(voxels)}")
        print("-" * 30)

File: chair_01.npy
  - Shape: (32, 32, 32)
  - Dtype: uint8
  - Min Value: 0
  - Max Value: 1
  - Unique Values: [0 1]
------------------------------
File: chair_08.npy
  - Shape: (32, 32, 32)
  - Dtype: uint8
  - Min Value: 0
  - Max Value: 1
  - Unique Values: [0 1]
------------------------------
File: chair_05.npy
  - Shape: (32, 32, 32)
  - Dtype: uint8
  - Min Value: 0
  - Max Value: 1
  - Unique Values: [0 1]
------------------------------
File: chair_05.npy
  - Shape: (32, 32, 32)
  - Dtype: uint8
  - Min Value: 0
  - Max Value: 1
  - Unique Values: [0 1]
------------------------------
File: chair_09.npy
  - Shape: (32, 32, 32)
  - Dtype: uint8
  - Min Value: 0
  - Max Value: 1
  - Unique Values: [0 1]
------------------------------


In [7]:
#Clear the background graph to prevent "Variable already exists" 
tf.compat.v1.reset_default_graph()
checkpoint_path = "/Users/nehir/Desktop/psvh_model_weights/23501.cptk"

# 2. Define Model and Tensors
# v_after shape: [1, 32, 32, 32]
v_before, v_after, img_ph = real_model()
gt_ph = tf.compat.v1.placeholder(tf.float32, [None, 32, 32, 32], name="vox_gt")
mask_ph = tf.compat.v1.placeholder(tf.float32, [None, 32, 32], name="mask_gt")

# --- MASK CONSISTENCY LOSS ---
# Project the 3D prediction into 2D by taking the max along the Z-axis
# This simulates a "silhouette" of the predicted voxels
v_projection = tf.reduce_max(v_after, axis=2) 

# Loss 1: 3D Voxel MSE
loss_voxel = tf.reduce_mean(tf.square(v_after - gt_ph))

# Loss 2: 2D Mask MSE (The paper's logic)
loss_mask = tf.reduce_mean(tf.square(v_projection - mask_ph))

# Combined Loss (You can weigh these, e.g., mask might need a higher weight)
total_loss = loss_voxel + (1.0 * loss_mask)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(total_loss)

# --- CRITICAL FIX: VARIABLE MANAGEMENT ---

# Identify which variables belong to the MODEL (to be restored) 
# and which belong to the OPTIMIZER (to be initialized)
all_vars = tf.compat.v1.global_variables()
model_vars = [v for v in all_vars if 'Adam' not in v.name and 'beta' not in v.name]
optimizer_vars = [v for v in all_vars if 'Adam' in v.name or 'beta' in v.name]

# Create saver for the model weights ONLY
saver = tf.compat.v1.train.Saver(var_list=model_vars)

# 4. Session Execution
sess = tf.compat.v1.Session()

# Initialize ONLY the new optimizer variables (so we don't overwrite the model)
if optimizer_vars:
    sess.run(tf.compat.v1.variables_initializer(optimizer_vars))
else:
    # Fallback if no optimizer vars found yet
    sess.run(tf.compat.v1.global_variables_initializer())

print(f"📡 Restoring weights from {checkpoint_path}...")
try:
    saver.restore(sess, checkpoint_path)
    print("✅ Weights Loaded Successfully")
except Exception as e:
    print(f"❌ Restore failed: {e}")
    print("Hint: Check if the checkpoint path is correct or if model architecture changed.")


pool1 Tensor("voxel/encoder/MaxPool:0", shape=(1, 64, 64, 96), dtype=float32)
pool2 Tensor("voxel/encoder/MaxPool_1:0", shape=(1, 32, 32, 128), dtype=float32)
pool3 Tensor("voxel/encoder/MaxPool_2:0", shape=(1, 16, 16, 256), dtype=float32)
pool4 Tensor("voxel/encoder/MaxPool_3:0", shape=(1, 8, 8, 256), dtype=float32)
pool5 Tensor("voxel/encoder/MaxPool_4:0", shape=(1, 4, 4, 256), dtype=float32)
pool6 Tensor("voxel/encoder/Mean:0", shape=(1, 256), dtype=float32)
feature Tensor("voxel/encoder/MatMul:0", shape=(1, 1024), dtype=float32)
feature_map Tensor("voxel/encoder/MaxPool_5:0", shape=(1, 2, 2, 256), dtype=float32)
Tensor("voxel/encoder/MatMul:0", shape=(1, 1024), dtype=float32)
Tensor("voxel/decoder/Reshape:0", shape=(1, 1, 1, 1, 1024), dtype=float32)
d1 Tensor("voxel/decoder/Relu:0", shape=(1, 4, 4, 4, 256), dtype=float32)
d2 Tensor("voxel/decoder/Add:0", shape=(1, 8, 8, 8, 128), dtype=float32)
d3 Tensor("voxel/decoder/Add_1:0", shape=(1, 16, 16, 16, 64), dtype=float32)
d4 Tensor("v

2026-01-24 22:01:46.580412: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2026-01-24 22:01:46.580434: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2026-01-24 22:01:46.580441: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.88 GB
2026-01-24 22:01:46.580457: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-24 22:01:46.580465: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2026-01-24 22:01:46.605895: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2026-01-24 22:01:46.625891: I tensorflow/core/gra

📡 Restoring weights from /Users/nehir/Desktop/psvh_model_weights/23501.cptk...
INFO:tensorflow:Restoring parameters from /Users/nehir/Desktop/psvh_model_weights/23501.cptk
✅ Weights Loaded Successfully


In [8]:
# Check a specific weight to see if it's actually loaded or just random
test_var = [v for v in tf.compat.v1.global_variables() if "wd00" in v.name][0]
weight_values = sess.run(test_var)

print(f"Variable Name: {test_var.name}")
print(f"Weight Mean: {np.mean(weight_values)}")
print(f"Weight Max: {np.max(weight_values)}")

if np.abs(np.mean(weight_values)) < 1e-7:
    print("❌ ERROR: Weights are all zeros or uninitialized!")
else:
    print("✅ Weights appear to be loaded.")

Variable Name: voxel/encoder/wd00:0
Weight Mean: 1.9196944776922464e-05
Weight Max: 0.10852489620447159
✅ Weights appear to be loaded.


In [9]:
# 1. Update the Placeholder to handle dynamic batch sizes
# (If your real_model() fixed the batch at 1, we need to redefine img_ph)
# Assuming img_ph was created inside real_model, ensure it supports batch size 4.

# 2. Create a NEW saver for saving the WHOLE state (Model + Optimizer)
# This is different from the restoration saver we used earlier
final_saver = tf.compat.v1.train.Saver() 

print("🚀 Starting Fine-tuning on M4 Pro...")

for epoch in range(EPOCHS):
    epoch_loss = 0
    for step in range(STEPS_PER_EPOCH):
        try:
            batch_imgs, batch_gt, batch_masks = next(gen)
            
            # Run optimization with mask_ph
            _, l = sess.run([optimizer, total_loss], feed_dict={
                img_ph: batch_imgs,
                gt_ph: batch_gt,
                mask_ph: batch_masks
            })
            
            epoch_loss += l
            
            if (step + 1) % 20 == 0:
                print(f"  Step {step+1}/{STEPS_PER_EPOCH} | Current Loss: {l:.5f}")
                
        except Exception as e:
            print(f"❌ Error at Step {step}: {e}")
            continue

    avg_loss = epoch_loss / STEPS_PER_EPOCH
    print(f"✅ Epoch {epoch+1} Complete | Avg Loss: {avg_loss:.5f}")

# 3. Save the result
final_saver.save(sess, save_path)
print(f"💾 Model saved to {save_path}")

🚀 Starting Fine-tuning on M4 Pro...
  Step 20/200 | Current Loss: 0.42615
  Step 40/200 | Current Loss: 0.15236
  Step 60/200 | Current Loss: 0.18149
  Step 80/200 | Current Loss: 0.13451
  Step 100/200 | Current Loss: 0.10590
  Step 120/200 | Current Loss: 0.13223
  Step 140/200 | Current Loss: 0.14896
  Step 160/200 | Current Loss: 0.39631
  Step 180/200 | Current Loss: 0.53719
  Step 200/200 | Current Loss: 0.27250
✅ Epoch 1 Complete | Avg Loss: 0.18967
  Step 20/200 | Current Loss: 0.10546
  Step 40/200 | Current Loss: 0.11678
  Step 60/200 | Current Loss: 0.13960
  Step 80/200 | Current Loss: 0.15170
  Step 100/200 | Current Loss: 0.35908
  Step 120/200 | Current Loss: 0.53401
  Step 140/200 | Current Loss: 0.13568
  Step 160/200 | Current Loss: 0.48854
  Step 180/200 | Current Loss: 0.20769
  Step 200/200 | Current Loss: 0.24869
✅ Epoch 2 Complete | Avg Loss: 0.17056
  Step 20/200 | Current Loss: 0.15477
  Step 40/200 | Current Loss: 0.71650
  Step 60/200 | Current Loss: 0.51945


In [178]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_voxels(voxels):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Render the cubes (True values)
    ax.voxels(voxels, edgecolor='k', facecolors='coral', alpha=0.7)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z') 
    plt.title("Fine-tuned PSVH Chair Result")
    plt.show()


In [7]:
def restore_model():
    tf.reset_default_graph()
    v_before, v_after, img_ph = real_model() # Define the "map"
    
    sess = tf.Session()
    
    # 1. Initialize EVERYTHING (Sets the stage)
    sess.run(tf.global_variables_initializer()) 
    
    # 2. Restore your FINE-TUNED weights (This is the important part)
    saver = tf.train.Saver()
    saver.restore(sess, fine_tuned_model_path)
    print("✅ Fine-tuned weights are now active in the session!")
    return sess, v_after, img_ph
